In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import sys
import numpy as np
import pandas as pd
# from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from tensorflow.keras.layers import BatchNormalization, Layer
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform


from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

from siamese import make_oneshot_task, test_oneshot

import numpy.random as rng
import pickle

In [ ]:
def get_pairwise_batch(batch_size, train_data):
    """
    Create batch of n pairs, half same class, half different class
    """
    n_classes, n_examples, w, h, d = train_data.shape
    

    rng = np.random.default_rng()

    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,))
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, w, h, d)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = np.random.randint(0, n_examples)
        pairs[0][i,:,:,:] = train_data[category, idx_1].reshape(w, h, d)
        idx_2 = np.random.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + np.random.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = train_data[category_2,idx_2].reshape(w, h, d)
    
    return pairs, targets


In [ ]:
train_in = open("mini-imagenet-cache-train.pkl", "rb")
train = pickle.load(train_in)
ungrouped_Xtrain = train["image_data"]
val_in = open("mini-imagenet-cache-val.pkl", "rb")
val = pickle.load(val_in)
ungrouped_Xval = val["image_data"]
val_data = ungrouped_Xval.reshape([16, 600, 84, 84, 3])

In [ ]:
train_mean = ungrouped_Xtrain.mean(axis=(0,1,2)) 
train_std = ungrouped_Xtrain.std(axis=(0,1,2))

In [ ]:
ungrouped_Xtrain = ungrouped_Xtrain.astype('float32')
ungrouped_Xval = ungrouped_Xval.astype('float32')

In [ ]:

ungrouped_Xtrain[..., 0] -= train_mean[0]
ungrouped_Xtrain[..., 1] -= train_mean[1]
ungrouped_Xtrain[..., 2] -= train_mean[2]
ungrouped_Xtrain[..., 0] /= train_std[0]
ungrouped_Xtrain[..., 1] /= train_std[1]
ungrouped_Xtrain[..., 2] /= train_std[2]


ungrouped_Xval[..., 0] -= train_mean[0]
ungrouped_Xval[..., 1] -= train_mean[1]
ungrouped_Xval[..., 2] -= train_mean[2]
ungrouped_Xval[..., 0] /= train_std[0]
ungrouped_Xval[..., 1] /= train_std[1]
ungrouped_Xval[..., 2] /= train_std[2]



In [ ]:
train_data = ungrouped_Xtrain.reshape([64, 600, 84, 84, 3])
val_data = ungrouped_Xval.reshape([16, 600, 84, 84, 3])

### Loading the validation images into tensors

### Saving the validation tensors on disk

In [ ]:
def initialize_weights(shape, name=None, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [ ]:
def initialize_bias(shape, name=None, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [ ]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [ ]:
model = get_siamese_model((84, 84, 3))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 84, 84, 3)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 84, 84, 3)]  0           []                               
                                                                                                  
 sequential (Sequential)        (None, 4096)         10648896    ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 4096)         0           ['sequential[0][0]',         

In [ ]:
optimizer = Adam(learning_rate = 0.00001)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

### Loading the train tensors

In [ ]:
# Hyper parameters
evaluate_every = 500 # interval for evaluating on one-shot tasks\
# download_every = 1000
batch_size = 64
n_iter = 10000 # No. of training iterations
N_way = 10 # how many classes for testing one-shot tasks. has to be less than num classes in dataset
n_val = 250 # how many one-shot tasks to validate on

# used for straight validation testing
val_batch_size = 64
epochs= 10
steps_per_epoch = 1000

best = -1

In [ ]:
def data_generator(data, batch_size):
    while True:
        (inputs,targets) = get_pairwise_batch(batch_size, train_data)
        yield inputs, targets

In [ ]:
def valid_data(data, batch_size):
    (inputs,targets) = get_pairwise_batch(batch_size, train_data)
    return inputs, targets

In [ ]:
history = model.fit(
    data_generator(train_data, batch_size),
    steps_per_epoch=steps_per_epoch,
    validation_data = data_generator(val_data, val_batch_size),
    validation_steps = 10,
    epochs=epochs, verbose=True)

Epoch 1/10
1000/1000 [==============================] - 177s 177ms/step - loss: 1.2244 - val_loss: 1.0191
Epoch 2/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.9055 - val_loss: 0.8378
Epoch 3/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.7921 - val_loss: 0.7639
Epoch 4/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.7443 - val_loss: 0.7254
Epoch 5/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.7239 - val_loss: 0.7123
Epoch 6/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.7106 - val_loss: 0.6931
Epoch 7/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.7020 - val_loss: 0.7131
Epoch 8/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.6963 - val_loss: 0.7001
Epoch 9/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.6916 - val_loss: 0.6922
Epoch 10/10
1000/1000 [=======================

In [ ]:
model.save_weights(os.path.join('./weights/', 'weights_binarycrossentropy_loss_2.h5'))

In [ ]:
model_path = './weights/'
model_name = "binarycrossentropy_loss_1"